In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
np.random.seed(3)

In [ ]:
df = pd.read_csv(r'/kaggle/input/human-activity-recognition-with-smartphones/train.csv')
df = df.sample(len(df)).reset_index(drop=True)
df.drop('subject',axis=1,inplace = True)

In [ ]:
df.Activity.value_counts()

In [ ]:
df.Activity = df.Activity.astype('category').cat.codes

In [ ]:
X=df.drop('Activity',axis=1)
y = df['Activity']
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [ ]:
def build_model(n_hidden=1,n_neurons=50,optimizer='RMSprop'):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=X_train.shape[1:]))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='relu'))
        model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.Dense(6,activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    return model

In [ ]:
keras_model = keras.wrappers.scikit_learn.KerasClassifier(build_model)
history = keras_model.fit(X_train,y_train,epochs=150,validation_data=(X_valid, y_valid),
                callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(22, 10))
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
param_distribs = {'n_hidden':(1,2,3,4),
                  'n_neurons':(50,100,150,200,250),
                  'optimizer': ['RMSprop','Adam','SGD']
                  }
rnd = RandomizedSearchCV(keras_model,param_distribs, n_iter=10, cv=3)
rnd.fit(X_train,y_train,epochs=100,validation_data=(X_valid,y_valid),
       callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
print(rnd.best_params_)
print(rnd.best_score_)

In [ ]:
model = rnd.best_estimator_.model
model.summary()

In [ ]:
test = pd.read_csv(r'/kaggle/input/human-activity-recognition-with-smartphones/test.csv')
test.drop('subject',axis=1,inplace = True)
test.Activity = test.Activity.astype('category').cat.codes
test_X = test.drop('Activity',axis=1)
test_X = scaler.transform(test_X)
test_y = test['Activity']
model.evaluate(test_X,test_y)

In [ ]:
y_pred=model.predict_classes(test_X)
print(classification_report(test_y,y_pred))